In [3]:
# Librerias generales
import pandas as pd
import numpy as np
import json

# Time
import datetime as dt
from datetime import datetime, timedelta

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt
from dython.model_utils import roc_graph
from dython.nominal import associations
%matplotlib inline

# OneHotEncoding
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2 # for chi-squared feature selection
import prince

from sklearn import metrics

import os
import json
import joblib

from pandas.io.json import json_normalize
from datetime import datetime 
from tqdm import tqdm
from sklearn.preprocessing import PowerTransformer
import umap
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy import stats

from sklearn.cluster import KMeans
from kmodes.kprototypes import KPrototypes


from lightgbm import LGBMClassifier
import shap
from sklearn.model_selection import cross_val_score


# Algoritmos
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import preprocessing 
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth

# Pipeline
from sklearn.pipeline import make_pipeline

from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

# Configuración de pandas 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

ModuleNotFoundError: No module named 'dython'

In [2]:
# new_data = Processing_data(data)
# new_data_cat = Processing_data(data_cath)
# Merge manual 

In [3]:
# 1. Leo data numerica
# 2. Leo data categorica
# 3. new_data = Processing_data(data, cath_data)
# 4. Ejecuta - merge cuando yo le diga

In [4]:
# Resultado

In [32]:
class Processing_data():
    """**Processing data**
    
    | When it comes to developing a Sempli cluster analyse we can use this class to do the work. To effectively develop a study you should know that this project comes with two steps:
    * Cleaning and executing the model
    * Testing the model
    
    | That being said, we are going to explain the steps to succesfully run our program:
    
    **Cleaning and executing the model**
    
    | The first step that you should do is read the csv or excel file providing the information (You can do that using the pandas library, and the methods read_csv or read_excel). In our case we have two provided datasets with finantial and cathegorical 
    information.
    
    | After you read those two files, and store the information in two different variables (data and cat_data) it's time to instance a variable with our class.
    
    **Example:**
    
        var_name = Procesing_data(data, cat_data, cleaning=True)
    
    | On the example provided we pass the two datasets (data & cat_data) and we pass a cleaning boolean to tell the class to clean that data in order to be process. The steps to process the data are the following:
    
    | Processing data steps:
    
        1- After reading the file (excel or csv) fill n/a rows of the file on json format rows using -fill_na_json method-
        
        2- Clean the dataframe to have the first draft of you desired output using -clean_data method-
        
        3- Delete unnecessary columns on numerical data using -delete_unnecessary_columns method-
        
        4- Organize the dataframe to have you desired dataframe to study using -organizing_data-
        
        5- Check the cathegorical dataset to use only the important information on it using -checking_cat_data method-
        
        6- We do a first merge to only analyze clients withing the two dataframes using -merge_two_tables method-
        
        7- After checking the clients present in the datasets we delete the cathegorical information that would be added later using -deleting_cat_info method-
        
        8- At this point we use a method called -customer_level- to analyze the behaviour of clients on their six latest payment
        
        9- When we have the finantial information clean and without any error we merge the two tables againg using the same method
        of step 6
    
    | All of these steps are present on the data_cleaning method. It's imperative to clean the data using these steps provided below to avoid any confusion with the clients information.
    
    | After doing all the data cleaning you can execute the model using the execute_model method. By executing this method you don't have to worry about cleaning the data because it takes care of it but you need to be aware of telling the class to clean the datasets by passing True to the **cleaning** variable of the constructor.
    
    | After you executing the model you can save it by using the save_model method and you will create a **testeo.joblib** file which you will use on the next step that is testing the model.
    
    **Testing the model**
    
    | Once you have the .joblib file correctly saved it's time to test the model and see it's behaviour.
    
    | To do so, you should instance a new variable with no cleaning option since your data is already clean.
    
    | For example:
    
        data_model = Processing_data(numeric, cathegorical, cleaning=False)
        
    | By doing this you'll tell the class to train and test the model.
    
    | Here it's an example on how to run the class: 
        **Cleaning and executing the model**
        
        >>> path_file = "loan_schedules_2.xlsx"
        >>> cols = ['client_id',
                'loan_schedule_id',
                'loan_id',
                'index',
                'expected_date',
                'state',
                'arrears_days',
                'expected_detail',
                'paid_detail']
        >>> data = pd.read_excel(path_file, sheet_name = 0, header = 0, usecols = cols).replace('NaT', np.nan)

        >>> path_file = "data_sempli_holberton.xlsx"
        >>> cat_cols = ['ID Cliente', 'Monto Acumulado', 'Uso de los recursos', 'Plazo',
                    'Sector', 'Ingresos', 'Ubicación', 'Estrato Mínimo',
                    'Procesos judiciales', 'Alertas', 'Score Bureau Empresa', 'Huellas de Consulta', 'Tiempo en el negocio',
                    'Website empresa', 'Instagram empresa', 'LinkedIn empresa',
                    'LinkedIn empresarios', 'Edad empresarios', 'Activador', 'Número de accionistas',
                    'Impacto', 'Acceso previso a la banca', '# Empleados',
                    'Mujeres empresarias', 'Mujeres en cargos directivos']
        >>> cat_data = pd.read_excel(path_file, sheet_name = 0, header = 2, usecols = cat_cols).replace('SIN INFO', np.nan)
        >>> cat_data = cat_data.dropna()
        
        >>> new_data = Processing_data(data, cat_data, cleaning=True)
        
        >>> new_data.execute_model()
        
        >>> new_data.execute_model()
        
        **Testing the model**
        
        # Generating aleatory sample of 20 clients
        >>> new_data = Processing_data(data, cat_data, cleaning=True)
        >>> data_prueba = new_data.data_cleaning()

        # Loading model
        >>> clf_model = joblib.load('testeo.joblib')

        # Generating aleatory sample
        >>> prueba= data_prueba.sample(7)
        >>> numerica = prueba.select_dtypes(exclude='object')
        >>> categorica = prueba.select_dtypes(exclude=['int64', 'float64'])

        #Procesar los datos de prueba
        >>> data_modelo = Processing_data(numerica, categorica, cleaning=False)
        
        >>> entrenar = data_modelo.reduction_dim().to_numpy()

        >>> print(f'clientes a evaluar {len(entrenar)}\n')
        >>> for i in range(len(entrenar)):
        >>>     print(f'cliente #{i} pertence al cluster: {clf_model.predict(entrenar, categorical=[2])[i]}')
        
    """

    def __init__(self, data, cat_data, cleaning=True):
        """Constructor of attributes
        
        | data = finaltial/numerical information of clients.
        | cat_data = cathegorical information about clients. Released by the risk department.
        
        """
        self.data = data
        self.kproto = []
        self.clusters = []
        self.cat_data = cat_data
        self.cleaning = cleaning

    def fill_na_json(self):
        """Filling missing values in json format columns"""
        col_names = ['expected_detail', 'paid_detail']
        for col_name in col_names:
            self.data[col_name] = self.data[col_name].apply(
                lambda x: "{}" if pd.isna(x) else x)
        return self.data

    @staticmethod
    def normalize_data(df, col_name):
        data = df[col_name].apply(json.loads)
        return pd.json_normalize(data)

    def clean_data(self):
        """
        Cleaning process of data:
        1- normalize
        2- deleting useless state (GRACE, PENDING, AND PARTIALLY PAID) on data
        """
        expected = Processing_data.normalize_data(self.data, 'expected_detail')
        paid = Processing_data.normalize_data(self.data, 'paid_detail')
        self.data = self.data.join(expected).join(paid, rsuffix='_paid')
        self.data = self.data[self.data.state != "GRACE"]
        self.data = self.data[self.data.state != "PENDING"]
        self.data = self.data[self.data.state != "PARTIALLY_PAID"]
        return self.data

    def delete_unnecessary_columns(self):
        """Deletes unnecesary columns produced by the json format"""
        data_droped_columns = ['penalty', 'interest', 'insurance', 'principal', 'taxRetentionValue',
                               'taxRetentionPercentage', 'legalCharge', 'preLegalCharge', 'feesPaid',
                               'fngTotal', 'fngValue',
                               'fngPercent', 'fngVatValue', 'fngVatPercent', 'monthlyDeferredInterest',
                               'penalty_paid', 'interest_paid', 'insurance_paid',
                               'principal_paid', 'taxRetentionValue_paid',
                               'taxRetentionPercentage_paid', 'legalCharge_paid',
                               'preLegalCharge_paid', 'feesPaid_paid', 'fngTotal_paid',
                               'fngValue_paid', 'fngPercent_paid', 'fngVatValue_paid',
                               'fngVatPercent_paid', 'monthlyDeferredInterest_paid', 'expected_detail', 'paid_detail']
        self.data = self.data.drop(columns=data_droped_columns)
        return self.data

    def organizing_data(self):
        """
        Organize data after having the first draft of your dataframe and fills total na rows with 0
        and groups clients by their six newest loan_id
        This is step three
        """
        self.data = self.data.sort_values(
            ['client_id', 'loan_id', 'expected_date'])
        self.data['total'] = self.data['total'].fillna(0)
        self.data['total_paid'] = self.data['total_paid'].fillna(0)
        self.data = self.data.groupby('loan_id').filter(
            lambda x: x['loan_id'].value_counts() > 6).groupby('loan_id').tail(6)
        return self.data

    def checking_cat_data(self):
        """Checks for the clients that are present in the two tables"""
        self.cat_data = self.cat_data.sort_values(
            by='ID Cliente', ascending=True)

        rep_columns = ['Procesos judiciales', 'Alertas', 'Website empresa',
                       'Instagram empresa', 'LinkedIn empresa',
                       'LinkedIn empresarios', 'Impacto', 'Acceso previso a la banca',
                       'Mujeres empresarias']

        # Replace values for Si/No
        self.cat_data[rep_columns] = self.cat_data[rep_columns].replace({'No procesos judiciales': 'No',
                                                                         'Sí procesos judiciales': 'Si',
                                                                         'No Alertas': 'No',
                                                                         'Sí Alertas': 'Si',
                                                                         'No website': 'No',
                                                                         'Si website': 'Si',
                                                                         'No Ig': 'No',
                                                                         'Si Ig': 'Si',
                                                                         'No LinkedIn': 'No',
                                                                         'Si LinkedIn': 'Si',
                                                                         'No LinkedIn empresarios': 'No',
                                                                         'Si LinkedIn empresarios': 'Si',
                                                                         'Si Impacto': 'Si',
                                                                         'No Impacto': 'No',
                                                                         'Si acceso a la banca': 'Si',
                                                                         'No acceso a la banca': 'No',
                                                                         'No mujeres empresarias': 'No',
                                                                         'Si mujeres empresarias': 'Si'})
        return self.cat_data

    def deleting_cat_info(self):
        """Deletes cat_data to analize clients on mean"""
        cat_cols = ['ID Cliente', 'Monto Acumulado', 'Uso de los recursos', 'Plazo',
                    'Sector', 'Ingresos', 'Ubicación', 'Estrato Mínimo',
                    'Procesos judiciales', 'Alertas', 'Score Bureau Empresa', 'Huellas de Consulta', 'Tiempo en el negocio',
                    'Website empresa', 'Instagram empresa', 'LinkedIn empresa',
                    'LinkedIn empresarios', 'Edad empresarios', 'Activador', 'Número de accionistas',
                    'Impacto', 'Acceso previso a la banca', '# Empleados',
                    'Mujeres empresarias', 'Mujeres en cargos directivos']

        self.data = self.data.drop(columns=cat_cols)
        return self.data

    def merge_two_tables(self):
        """Merging two tables: Numerical and cathegorical"""
        self.data = self.data.merge(
            self.cat_data, left_on='client_id', right_on='ID Cliente')
        return self.data

    def customer_level(self):
        """Compress dataframe into a better study"""
        self.data = self.data.groupby(['client_id', 'loan_id']).agg({
            'state': lambda x:  x.value_counts().index[0],
            #'state': lambda x: x.iloc[-1],  # Devuelve el último state
            'arrears_days': 'sum',
            'total_paid': 'sum'}).round(0)  # Se suman todas las facturas
        self.data['state'] = self.data['arrears_days'].apply(lambda x: 'LATE' if x > 16 else 'PAID')
        self.data.reset_index(inplace=True)
        return self.data

    def head(self):
        """print head of df"""
        return self.data.head()

    # Funcion compacta de limpieza de data
    def data_cleaning(self):
        """This function resumes all the steps of organizing and cleaning the datasets"""
        self.data = self.fill_na_json()
        self.data = self.clean_data()
        self.data = self.delete_unnecessary_columns()
        self.data = self.organizing_data()
        self.cat_data = self.checking_cat_data()
        self.data = self.merge_two_tables()
        self.data = self.deleting_cat_info()
        self.data = self.customer_level()
        self.data = self.merge_two_tables()
        return self.data
        
# ---------------------------------------

    def transform_data(self):
        """transform_data : Transforma los datos numericos con el metodo PowerTransformer"""
        if self.cleaning == True:
            self.data = self.data_cleaning()
        #else:
        #    self.cat_data["ID Cliente"] = self.data['client_id']
        #    self.data.drop(["ID Cliente"], axis= 1, inplace=True)
        #    self.data = self.merge_two_tables()
        self.data.drop(["client_id", 'loan_id'], axis= 1, inplace=True)
        for c in self.data.select_dtypes(exclude='object').columns:
            pt = PowerTransformer()
            self.data[c] = pt.fit_transform(
                np.array(self.data[c]).reshape(-1, 1))
        return self.data

    def reduction_dim(self):
        """reduction: Reduce la dimensionalidad de los datos aplicando Analisis Factorial de Datos Mixtos(FAMD)"""
        self.data = self.transform_data()
        
        self.data['state'] = self.data['state'].replace(to_replace="LATE",value="0")
        self.data['state'] = self.data['state'].replace(to_replace="PAID",value="1")
        self.data['state'] = self.data['state'].astype(object)

        # Declarar metodo para aplicar FAMD
        famd = prince.FAMD(
            n_components=2,
            n_iter=3,
            copy=True,
            check_input=True,
            engine='auto',
            random_state=42)

        # Ajustar y transformar la dimensión aplicando FAMD
        famd = famd.fit(self.data)
        transformada = famd.transform(self.data)

        Y = transformada.to_numpy()
        principalDf_famd = pd.DataFrame(
            data=Y, columns=['principal component 1', 'principal component 2'])
        finalDf_Cat_famd = pd.concat(
            [principalDf_famd, self.data['state']], axis=1, ignore_index=True)

        self.data = finalDf_Cat_famd
        return self.data

    def execute_model(self):
        """execute: Función que ejecuta el modelo con los datos procesados"""
        self.data = self.reduction_dim().to_numpy()
        self.kproto = KPrototypes(n_clusters=2, init = 'Huang', max_iter=100, verbose = 1,
                                  n_init = 15, random_state=4444, n_jobs=-1, gamma=.25)
        self.clusters = self.kproto.fit_predict(self.data, categorical=[2])
        #self.kproto = kproto
        #print(self.kproto)
        #print(self.clusters)
        return self.clusters

    def guardar_model(self):
        joblib.dump(self.execute_model(), 'testeo.joblib')
        print("Se ha guardado correctamente!")

In [6]:
#------------------------------------------------------------------

In [4]:
path_file = "loan_schedules_2.xlsx"
cols = ['client_id',
        'loan_schedule_id',
        'loan_id',
        'index',
        'expected_date',
        'state',
        'arrears_days',
        'expected_detail',
        'paid_detail']
data = pd.read_excel(path_file, sheet_name = 0, header = 0, usecols = cols).replace('NaT', np.nan)

In [5]:
path_file = "data_sempli_holberton.xlsx"
cat_cols = ['ID Cliente', 'Monto Acumulado', 'Uso de los recursos', 'Plazo',
       'Sector', 'Ingresos', 'Ubicación', 'Estrato Mínimo',
       'Procesos judiciales', 'Alertas', 'Score Bureau Empresa', 'Huellas de Consulta', 'Tiempo en el negocio',
       'Website empresa', 'Instagram empresa', 'LinkedIn empresa',
       'LinkedIn empresarios', 'Edad empresarios', 'Activador', 'Número de accionistas',
       'Impacto', 'Acceso previso a la banca', '# Empleados',
       'Mujeres empresarias', 'Mujeres en cargos directivos']
cat_data = pd.read_excel(path_file, sheet_name = 0, header = 2, usecols = cat_cols).replace('SIN INFO', np.nan)
cat_data = cat_data.dropna()

In [39]:
# Instanciar la clase para etapa de limpieza y ejcutar el modelo
new_data = Processing_data(data, cat_data, cleaning=True)

In [34]:
new_data.transform_data()

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


,state,arrears_days,total_paid,ID Cliente,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
0,LATE,0.068201,1.990516,-2.323541,0.247658,CR,25 a 36 meses,Servicios,Seed,Medellín,0.481027,No,No,0.592443,0.514087,0.824309,Si,Si,Si,Si,0.542674,Internet,0.017484,No,Si,0.176367,No,0.447494
1,LATE,0.219112,1.308246,-2.323541,0.247658,CR,25 a 36 meses,Servicios,Seed,Medellín,0.481027,No,No,0.592443,0.514087,0.824309,Si,Si,Si,Si,0.542674,Internet,0.017484,No,Si,0.176367,No,0.447494
2,PAID,-0.027869,-0.515458,-2.033775,-1.405435,KT - EX,13 a 24 meses,Servicios,Seed,Medellín,-1.519614,No,No,0.585245,-0.688637,0.456239,Si,No,Si,No,-0.942694,SocialNetworks,0.736474,No,Si,-1.700640,No,-1.459134
3,LATE,0.735415,-0.901116,-1.901462,-1.626569,SP,13 a 24 meses,Industria,Seed,Medellín,-0.523889,No,No,0.145027,-1.130869,0.638175,Si,No,Si,Si,0.451404,CreditProfessionalContact,0.017484,No,Si,-1.107593,No,-1.459134
4,LATE,0.735415,-0.946665,-1.901462,-1.626569,SP,13 a 24 meses,Industria,Seed,Medellín,-0.523889,No,No,0.145027,-1.130869,0.638175,Si,No,Si,Si,0.451404,CreditProfessionalContact,0.017484,No,Si,-1.107593,No,-1.459134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,LATE,0.335600,-0.475159,1.951817,-1.405435,KT - CO,13 a 24 meses,Servicios,Seed,Bogotá D.C.,0.481027,No,No,0.900597,-0.688637,1.014737,Si,Si,No,Si,-0.054151,SocialNetworks,0.017484,No,Si,-0.896798,No,-1.459134
573,PAID,-0.613883,0.238679,2.032265,0.012235,KT - EX,25 a 36 meses,Servicios,Venture,Bogotá D.C.,-0.523889,No,Si,0.358389,0.069432,0.278411,Si,No,Si,Si,-0.166537,CommunicationMedia,-1.311581,No,Si,0.295523,No,0.867677
574,LATE,0.676864,-0.151688,2.040592,-0.436514,CR,25 a 36 meses,Servicios,Venture,Bogotá D.C.,-0.523889,No,No,0.492277,-0.688637,0.824309,No,No,No,No,-0.530803,Internet,0.736474,Si,Si,0.038432,No,1.411840
575,PAID,-1.494842,1.209378,2.084151,0.750264,KT - EX,25 a 36 meses,Servicios,Venture,Medellín,1.493650,No,Si,0.556522,-0.688637,-0.065288,Si,Si,No,Si,-0.166537,SocialNetworks,0.017484,Si,Si,-0.124416,No,2.024831


In [35]:
excel_final = pd.read_csv('data_prueba.csv')
excel_final['state'].value_counts()

PAID    293
LATE    284
Name: state, dtype: int64

In [37]:
%%time
cluster = new_data.execute_model()
cluster

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


Best run was number 3
Wall time: 15.9 s


array([1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,

In [38]:
cluster

array([1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,

In [45]:
excel_final['cluster'] = cluster
#clusters_kproto_pca.reset_index(inplace=True)
#excel_final.drop(['index', 'level_0'],axis=1, inplace=True)
excel_final.head()

,client_id,loan_id,state,arrears_days,total_paid,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos,cluster
0,241,1076,LATE,17,147915805.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925,8,Si,Si,Si,Si,45,Internet,2,No,Si,16,No,2,1
1,241,1244,LATE,23,91852726.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925,8,Si,Si,Si,Si,45,Internet,2,No,Si,16,No,2,1
2,362,1469,PAID,14,15211138.0,50000000,KT - EX,13 a 24 meses,Servicios,Seed,Medellín,3,No,No,924,2,Si,No,Si,No,32,SocialNetworks,3,No,Si,3,No,0,0
3,428,119,LATE,64,8713879.0,40000000,SP,13 a 24 meses,Industria,Seed,Medellín,4,No,No,861,1,Si,No,Si,Si,44,CreditProfessionalContact,2,No,Si,5,No,0,0
4,428,120,LATE,64,8102608.0,40000000,SP,13 a 24 meses,Industria,Seed,Medellín,4,No,No,861,1,Si,No,Si,Si,44,CreditProfessionalContact,2,No,Si,5,No,0,0


In [47]:
excel_final['cluster'].value_counts()

0    297
1    280
Name: cluster, dtype: int64

In [48]:
excel_final.to_csv('propuesta_cluster.csv',encoding='utf-8-sig')

In [77]:
%%time
new_data.guardar_model()

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


Best run was number 3
KPrototypes(gamma=0.25, n_clusters=2, n_init=15, n_jobs=-1, random_state=4444,
            verbose=1)
Se ha guardado correctamente!
Wall time: 17 s


#### Testeando modelo

In [78]:
# Se genera datos para prueba
new_data = Processing_data(data, cat_data, cleaning=True)
data_prueba = new_data.data_cleaning()

# Cargar modelo
clf_model = joblib.load('testeo.joblib')

# Generando un muestra aleatoria de 20 clientes
prueba= data_prueba.sample(7)
numerica = prueba.select_dtypes(exclude='object')
categorica = prueba.select_dtypes(exclude=['int64', 'float64'])

#Procesar los datos de prueba
data_modelo = Processing_data(numerica, categorica, cleaning=False)
#numerica.drop(['client_id', 'loan_id'], axis=1, inplace=True)
entrenar = data_modelo.reduction_dim().to_numpy()

print(f'clientes a evaluar {len(entrenar)}\n')
for i in range(len(entrenar)):
    print(f'cliente #{i} pertence al cluster: {clf_model.predict(entrenar, categorical=[2])[i]}')


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
divide by zero encountered in log
overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


clientes a evaluar 7

cliente #0 pertence al cluster: 0
cliente #1 pertence al cluster: 1
cliente #2 pertence al cluster: 0
cliente #3 pertence al cluster: 0
cliente #4 pertence al cluster: 0
cliente #5 pertence al cluster: 1
cliente #6 pertence al cluster: 0


In [59]:
#categorica["ID Cliente"] = numerica ['client_id']
#numerica.drop(["ID Cliente"], axis= 1, inplace=True)
#categorica
    

In [8]:
new_data.transform_data()

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


,state,arrears_days,total_paid,ID Cliente,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
0,0.745155,0.068201,1.990516,-2.323541,0.247658,CR,25 a 36 meses,Servicios,Seed,Medellín,0.481027,No,No,0.592443,0.514087,0.824309,Si,Si,Si,Si,0.542674,Internet,0.017484,No,Si,0.176367,No,0.447494
1,0.745155,0.219112,1.308246,-2.323541,0.247658,CR,25 a 36 meses,Servicios,Seed,Medellín,0.481027,No,No,0.592443,0.514087,0.824309,Si,Si,Si,Si,0.542674,Internet,0.017484,No,Si,0.176367,No,0.447494
2,0.745155,-0.027869,-0.515458,-2.033775,-1.405435,KT - EX,13 a 24 meses,Servicios,Seed,Medellín,-1.519614,No,No,0.585245,-0.688637,0.456239,Si,No,Si,No,-0.942694,SocialNetworks,0.736474,No,Si,-1.700640,No,-1.459134
3,0.745155,0.735415,-0.901116,-1.901462,-1.626569,SP,13 a 24 meses,Industria,Seed,Medellín,-0.523889,No,No,0.145027,-1.130869,0.638175,Si,No,Si,Si,0.451404,CreditProfessionalContact,0.017484,No,Si,-1.107593,No,-1.459134
4,0.745155,0.735415,-0.946665,-1.901462,-1.626569,SP,13 a 24 meses,Industria,Seed,Medellín,-0.523889,No,No,0.145027,-1.130869,0.638175,Si,No,Si,Si,0.451404,CreditProfessionalContact,0.017484,No,Si,-1.107593,No,-1.459134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,-1.342003,0.335600,-0.475159,1.951817,-1.405435,KT - CO,13 a 24 meses,Servicios,Seed,Bogotá D.C.,0.481027,No,No,0.900597,-0.688637,1.014737,Si,Si,No,Si,-0.054151,SocialNetworks,0.017484,No,Si,-0.896798,No,-1.459134
573,-1.342003,-0.613883,0.238679,2.032265,0.012235,KT - EX,25 a 36 meses,Servicios,Venture,Bogotá D.C.,-0.523889,No,Si,0.358389,0.069432,0.278411,Si,No,Si,Si,-0.166537,CommunicationMedia,-1.311581,No,Si,0.295523,No,0.867677
574,-1.342003,0.676864,-0.151688,2.040592,-0.436514,CR,25 a 36 meses,Servicios,Venture,Bogotá D.C.,-0.523889,No,No,0.492277,-0.688637,0.824309,No,No,No,No,-0.530803,Internet,0.736474,Si,Si,0.038432,No,1.411840
575,0.745155,-1.494842,1.209378,2.084151,0.750264,KT - EX,25 a 36 meses,Servicios,Venture,Medellín,1.493650,No,Si,0.556522,-0.688637,-0.065288,Si,Si,No,Si,-0.166537,SocialNetworks,0.017484,Si,Si,-0.124416,No,2.024831


In [34]:
%%time
new_data.execute_model()

TypeError: invalid type promotion

In [16]:
new_data.data_cleaning()

KeyError: 'expected_detail'

In [271]:
new_data.head()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,expected_detail,paid_detail
0,1471,952,54,7,2019-02-27,PENDING,0,"{""total"": 5197196.73, ""penalty"": 0, ""interest""...","{""total"": 0, ""penalty"": 0, ""interest"": 0, ""ins..."
1,2442,13696,592,6,2019-03-18,PAID,0,"{""total"": 22872996.50, ""penalty"": 0, ""interest...","{""total"": 22872996.50, ""penalty"": 0, ""interest..."
2,9273,43776,1831,3,2020-03-04,PAID,5,"{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""...","{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""..."
3,2952,25418,1065,2,2019-02-12,PAID,0,"{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""...","{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""..."
4,2459,14548,628,13,2019-11-19,LATE,76,{},{}


In [272]:
new_data.fill_na_json()
print(new_data)

In [273]:
new_data.head()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,expected_detail,paid_detail
0,1471,952,54,7,2019-02-27,PENDING,0,"{""total"": 5197196.73, ""penalty"": 0, ""interest""...","{""total"": 0, ""penalty"": 0, ""interest"": 0, ""ins..."
1,2442,13696,592,6,2019-03-18,PAID,0,"{""total"": 22872996.50, ""penalty"": 0, ""interest...","{""total"": 22872996.50, ""penalty"": 0, ""interest..."
2,9273,43776,1831,3,2020-03-04,PAID,5,"{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""...","{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""..."
3,2952,25418,1065,2,2019-02-12,PAID,0,"{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""...","{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""..."
4,2459,14548,628,13,2019-11-19,LATE,76,{},{}


In [274]:
print(type(new_data))

<class '__main__.Processing_data'>


In [275]:
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [276]:
new_data.clean_data()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,expected_detail,paid_detail,total,penalty,interest,insurance,principal,taxRetentionValue,taxRetentionPercentage,fngTotal,fngValue,fngPercent,fngVatValue,legalCharge,fngVatPercent,preLegalCharge,feesPaid,monthlyDeferredInterest,total_paid,penalty_paid,interest_paid,insurance_paid,principal_paid,taxRetentionValue_paid,taxRetentionPercentage_paid,fngTotal_paid,fngValue_paid,fngPercent_paid,fngVatValue_paid,legalCharge_paid,fngVatPercent_paid,preLegalCharge_paid,feesPaid_paid,monthlyDeferredInterest_paid
1,2442,13696,592,6,2019-03-18,PAID,0,"{""total"": 22872996.50, ""penalty"": 0, ""interest...","{""total"": 22872996.50, ""penalty"": 0, ""interest...",22872996.50,0.0,358105.48,0.0,22514891.02,14324.2192,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22872996.50,0.0,358105.48,0.0,22514891.02,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9273,43776,1831,3,2020-03-04,PAID,5,"{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""...","{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""...",7322168.42,0.0,3015180.80,59878.0,4306987.62,120607.2320,0.04,0.0,0.0,0.028,0.0,NaN,0.19,NaN,NaN,NaN,7322168.42,0.0,3015180.80,0.0,4306987.62,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2952,25418,1065,2,2019-02-12,PAID,0,"{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""...","{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""...",4264953.71,0.0,1519955.30,29939.0,2744998.41,60798.2120,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4264953.71,0.0,1519955.30,0.0,2744998.41,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2459,14548,628,13,2019-11-19,LATE,76,{},{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4458,31214,1320,6,2019-11-27,PAID,0,"{""total"": 2598166.29, ""penalty"": 0, ""feesPaid""...","{""total"": 2598166.29, ""penalty"": 0, ""feesPaid""...",2598166.29,0.0,493134.76,10828.0,2105031.53,19725.3904,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2598166.29,0.0,493134.76,0.0,2105031.53,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12754,7450,38103,1609,5,2020-02-13,PAID,0,"{""total"": 1656810.57, ""penalty"": 0, ""feesPaid""...","{""total"": 1656810.57, ""penalty"": 0, ""feesPaid""...",1656810.57,0.0,663022.29,0.0,993788.28,0.0000,0.04,0.0,0.0,0.000,0.0,NaN,0.00,NaN,NaN,0.0,1656810.57,0.0,663022.29,0.0,993788.28,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
12755,7450,38104,1609,6,2020-03-13,PAID,0,"{""total"": 1656810.57, ""penalty"": 0, ""feesPaid""...","{""total"": 1656810.57, ""penalty"": 0, ""feesPaid""...",1656810.57,0.0,647022.30,0.0,1009788.27,0.0000,0.04,0.0,0.0,0.000,0.0,NaN,0.00,NaN,NaN,0.0,1656810.57,0.0,647022.30,0.0,1009788.27,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
12756,1068,24104,984,21,2020-03-09,LATE,22,{},{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12758,2454,15828,682,10,2019-08-08,PAID,0,"{""total"": 2772721.57, ""penalty"": 0, ""interest""...","{""total"": 2772721.57, ""penalty"": 0, ""interest""...",2772721.57,0.0,131385.57,0.0,2641336.00,5255.4228,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2772721.57,0.0,131385.57,0.0,2641336.00,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [277]:
#new_data.deleting_ocurrencys_state()
#new_data.head()

In [278]:
#new_data.normalize_expected()
#new_data.normalize_paid()
#new_data.head()

In [279]:
new_data.delete_unnecessary_columns()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,total,total_paid
1,2442,13696,592,6,2019-03-18,PAID,0,22872996.50,22872996.50
2,9273,43776,1831,3,2020-03-04,PAID,5,7322168.42,7322168.42
3,2952,25418,1065,2,2019-02-12,PAID,0,4264953.71,4264953.71
4,2459,14548,628,13,2019-11-19,LATE,76,NaN,NaN
5,4458,31214,1320,6,2019-11-27,PAID,0,2598166.29,2598166.29
...,...,...,...,...,...,...,...,...,...
12754,7450,38103,1609,5,2020-02-13,PAID,0,1656810.57,1656810.57
12755,7450,38104,1609,6,2020-03-13,PAID,0,1656810.57,1656810.57
12756,1068,24104,984,21,2020-03-09,LATE,22,NaN,NaN
12758,2454,15828,682,10,2019-08-08,PAID,0,2772721.57,2772721.57


In [280]:
new_data.organizing_data()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,total,total_paid
5677,14,80,14,6,2017-10-27,PAID,4,1497723.06,1497723.06
860,14,81,14,7,2017-11-27,PAID,0,1499416.03,1499416.03
861,14,82,14,8,2017-12-27,PAID,0,1497723.06,1497723.06
862,14,83,14,9,2018-01-29,PAID,0,1497723.06,1497723.06
863,14,84,14,10,2018-02-27,PAID,0,20078872.73,20078872.73
...,...,...,...,...,...,...,...,...,...
6763,7279,37729,1595,3,2019-12-02,PAID,0,3233354.83,3233354.83
6783,7279,37730,1595,4,2019-12-30,PAID,0,3233354.83,3233354.83
6786,7279,37731,1595,5,2020-01-30,PAID,0,3233354.83,3233354.83
6787,7279,37732,1595,6,2020-03-02,PAID,0,3233354.83,3233354.83


In [281]:
new_data.checking_cat_data()
new_data.merge_two_tables()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,total,total_paid,ID Cliente,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
0,241,25718,1076,8,2019-08-14,PAID,16,5947504.88,5947504.88,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
1,241,25719,1076,9,2019-09-16,PAID,1,5972929.73,5972929.73,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
2,241,25720,1076,10,2019-10-30,PAID,0,5941645.15,5941645.15,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
3,241,25721,1076,11,2019-11-29,PAID,0,5939653.40,5939653.40,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
4,241,25722,1076,12,2019-12-30,PAID,0,5939653.40,5939653.40,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3457,7279,37729,1595,3,2019-12-02,PAID,0,3233354.83,3233354.83,7279.0,50000000,KT - CO,13 a 24 meses,Servicios,Venture,Bogotá D.C.,3,No,No,226.0,8.0,1990,Si,No,No,Si,56,Referrer,2.0,No,Si,6,No,0
3458,7279,37730,1595,4,2019-12-30,PAID,0,3233354.83,3233354.83,7279.0,50000000,KT - CO,13 a 24 meses,Servicios,Venture,Bogotá D.C.,3,No,No,226.0,8.0,1990,Si,No,No,Si,56,Referrer,2.0,No,Si,6,No,0
3459,7279,37731,1595,5,2020-01-30,PAID,0,3233354.83,3233354.83,7279.0,50000000,KT - CO,13 a 24 meses,Servicios,Venture,Bogotá D.C.,3,No,No,226.0,8.0,1990,Si,No,No,Si,56,Referrer,2.0,No,Si,6,No,0
3460,7279,37732,1595,6,2020-03-02,PAID,0,3233354.83,3233354.83,7279.0,50000000,KT - CO,13 a 24 meses,Servicios,Venture,Bogotá D.C.,3,No,No,226.0,8.0,1990,Si,No,No,Si,56,Referrer,2.0,No,Si,6,No,0


In [282]:
new_data.deleting_cat_info()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,total,total_paid
0,241,25718,1076,8,2019-08-14,PAID,16,5947504.88,5947504.88
1,241,25719,1076,9,2019-09-16,PAID,1,5972929.73,5972929.73
2,241,25720,1076,10,2019-10-30,PAID,0,5941645.15,5941645.15
3,241,25721,1076,11,2019-11-29,PAID,0,5939653.40,5939653.40
4,241,25722,1076,12,2019-12-30,PAID,0,5939653.40,5939653.40
...,...,...,...,...,...,...,...,...,...
3457,7279,37729,1595,3,2019-12-02,PAID,0,3233354.83,3233354.83
3458,7279,37730,1595,4,2019-12-30,PAID,0,3233354.83,3233354.83
3459,7279,37731,1595,5,2020-01-30,PAID,0,3233354.83,3233354.83
3460,7279,37732,1595,6,2020-03-02,PAID,0,3233354.83,3233354.83


In [283]:
new_data.customer_level()

,client_id,loan_id,state,arrears_days,total_paid
0,241,1076,PAID,17,147915805.0
1,241,1244,PAID,23,91852726.0
2,362,1469,PAID,14,15211138.0
3,428,119,PAID,64,8713879.0
4,428,120,PAID,64,8102608.0
...,...,...,...,...,...
572,6734,1499,LATE,29,16038476.0
573,6982,1503,LATE,4,36277437.0
574,7008,1593,LATE,57,23829284.0
575,7145,1492,PAID,0,85184813.0


In [284]:
new_data.merge_two_tables()

,client_id,loan_id,state,arrears_days,total_paid,ID Cliente,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
0,241,1076,PAID,17,147915805.0,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
1,241,1244,PAID,23,91852726.0,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
2,362,1469,PAID,14,15211138.0,362.0,50000000,KT - EX,13 a 24 meses,Servicios,Seed,Medellín,3,No,No,924.0,2.0,2014,Si,No,Si,No,32,SocialNetworks,3.0,No,Si,3,No,0
3,428,119,PAID,64,8713879.0,428.0,40000000,SP,13 a 24 meses,Industria,Seed,Medellín,4,No,No,861.0,1.0,2015,Si,No,Si,Si,44,CreditProfessionalContact,2.0,No,Si,5,No,0
4,428,120,PAID,64,8102608.0,428.0,40000000,SP,13 a 24 meses,Industria,Seed,Medellín,4,No,No,861.0,1.0,2015,Si,No,Si,Si,44,CreditProfessionalContact,2.0,No,Si,5,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,6734,1499,LATE,29,16038476.0,6734.0,50000000,KT - CO,13 a 24 meses,Servicios,Seed,Bogotá D.C.,5,No,No,967.0,2.0,2017,Si,Si,No,Si,39,SocialNetworks,2.0,No,Si,6,No,0
573,6982,1503,LATE,4,36277437.0,6982.0,200000000,KT - EX,25 a 36 meses,Servicios,Venture,Bogotá D.C.,4,No,Si,892.0,5.0,2013,Si,No,Si,Si,38,CommunicationMedia,1.0,No,Si,18,No,3
574,7008,1593,LATE,57,23829284.0,7008.0,130000000,CR,25 a 36 meses,Servicios,Venture,Bogotá D.C.,4,No,No,911.0,2.0,2016,No,No,No,No,35,Internet,3.0,Si,Si,14,No,5
575,7145,1492,PAID,0,85184813.0,7145.0,400000000,KT - EX,25 a 36 meses,Servicios,Venture,Medellín,6,No,Si,920.0,2.0,2011,Si,Si,No,Si,38,SocialNetworks,2.0,Si,Si,12,No,9


In [1]:
class Processing_data():
    """**Processing data**
    
    | When it comes to developing a Sempli cluster analyse we can use this class to do the work. To effectively develop a study you should know that this project comes with two steps:
    * Cleaning and executing the model
    * Testing the model
    
    | That being said, we are going to explain the steps to succesfully run our program:
    
    **Cleaning and executing the model**
    
    | The first step that you should do is read the csv or excel file providing the information (You can do that using the pandas library, and the methods read_csv or read_excel). In our case we have two provided datasets with finantial and cathegorical 
    information.
    
    | After you read those two files, and store the information in two different variables (data and cat_data) it's time to instance a variable with our class.
    
    **Example:**
    
        var_name = Procesing_data(data, cat_data, cleaning=True)
    
    | On the example provided we pass the two datasets (data & cat_data) and we pass a cleaning boolean to tell the class to clean that data in order to be process. The steps to process the data are the following:
    
    | Processing data steps:
    
        1- After reading the file (excel or csv) fill n/a rows of the file on json format rows using -fill_na_json method-
        
        2- Clean the dataframe to have the first draft of you desired output using -clean_data method-
        
        3- Delete unnecessary columns on numerical data using -delete_unnecessary_columns method-
        
        4- Organize the dataframe to have you desired dataframe to study using -organizing_data-
        
        5- Check the cathegorical dataset to use only the important information on it using -checking_cat_data method-
        
        6- We do a first merge to only analyze clients withing the two dataframes using -merge_two_tables method-
        
        7- After checking the clients present in the datasets we delete the cathegorical information that would be added later using -deleting_cat_info method-
        
        8- At this point we use a method called -customer_level- to analyze the behaviour of clients on their six latest payment
        
        9- When we have the finantial information clean and without any error we merge the two tables againg using the same method
        of step 6
    
    | All of these steps are present on the data_cleaning method. It's imperative to clean the data using these steps provided below to avoid any confusion with the clients information.
    
    | After doing all the data cleaning you can execute the model using the execute_model method. By executing this method you don't have to worry about cleaning the data because it takes care of it but you need to be aware of telling the class to clean the datasets by passing True to the **cleaning** variable of the constructor.
    
    | After you executing the model you can save it by using the save_model method and you will create a **testeo.joblib** file which you will use on the next step that is testing the model.
    
    **Testing the model**
    
    | Once you have the .joblib file correctly saved it's time to test the model and see it's behaviour.
    
    | To do so, you should instance a new variable with no cleaning option since your data is already clean.
    
    | For example:
    
        data_model = Processing_data(numeric, cathegorical, cleaning=False)
        
    | By doing this you'll tell the class to train and test the model.
    
    | Here it's an example on how to run the class: 
        **Cleaning and executing the model**
        
        >>> path_file = "loan_schedules_2.xlsx"
        >>> cols = ['client_id',
                'loan_schedule_id',
                'loan_id',
                'index',
                'expected_date',
                'state',
                'arrears_days',
                'expected_detail',
                'paid_detail']
        >>> data = pd.read_excel(path_file, sheet_name = 0, header = 0, usecols = cols).replace('NaT', np.nan)

        >>> path_file = "data_sempli_holberton.xlsx"
        >>> cat_cols = ['ID Cliente', 'Monto Acumulado', 'Uso de los recursos', 'Plazo',
                    'Sector', 'Ingresos', 'Ubicación', 'Estrato Mínimo',
                    'Procesos judiciales', 'Alertas', 'Score Bureau Empresa', 'Huellas de Consulta', 'Tiempo en el negocio',
                    'Website empresa', 'Instagram empresa', 'LinkedIn empresa',
                    'LinkedIn empresarios', 'Edad empresarios', 'Activador', 'Número de accionistas',
                    'Impacto', 'Acceso previso a la banca', '# Empleados',
                    'Mujeres empresarias', 'Mujeres en cargos directivos']
        >>> cat_data = pd.read_excel(path_file, sheet_name = 0, header = 2, usecols = cat_cols).replace('SIN INFO', np.nan)
        >>> cat_data = cat_data.dropna()
        
        >>> new_data = Processing_data(data, cat_data, cleaning=True)
        
        >>> new_data.execute_model()
        
        >>> new_data.execute_model()
        
        **Testing the model**
        
        # Generating aleatory sample of 20 clients
        >>> new_data = Processing_data(data, cat_data, cleaning=True)
        >>> data_prueba = new_data.data_cleaning()

        # Loading model
        >>> clf_model = joblib.load('testeo.joblib')

        # Generating aleatory sample
        >>> prueba= data_prueba.sample(7)
        >>> numerica = prueba.select_dtypes(exclude='object')
        >>> categorica = prueba.select_dtypes(exclude=['int64', 'float64'])

        #Procesar los datos de prueba
        >>> data_modelo = Processing_data(numerica, categorica, cleaning=False)
        
        >>> entrenar = data_modelo.reduction_dim().to_numpy()

        >>> print(f'clientes a evaluar {len(entrenar)}\n')
        >>> for i in range(len(entrenar)):
        >>>     print(f'cliente #{i} pertence al cluster: {clf_model.predict(entrenar, categorical=[2])[i]}')
        
    """

    def __init__(self, data, cat_data):
        """Constructor of attributes
        
        | data = finaltial/numerical information of clients.
        | cat_data = cathegorical information about clients. Released by the risk department.
        | kproto = In this variable we have the trained model
        | clusters = Centroids of the trained algorithm
        | cleaning = Holds a boolean (True/False). True = clean the dataset and save the model, False= Test the model
        """
        self.data = data
        self.kproto = []
        self.clusters = []
        self.cat_data = cat_data
        #self.cleaning = cleaning

    def fill_na_json(self):
        """Filling missing values in json format columns"""
        col_names = ['expected_detail', 'paid_detail']
        for col_name in col_names:
            self.data[col_name] = self.data[col_name].apply(
                lambda x: "{}" if pd.isna(x) else x)
        return self.data

    @staticmethod
    def normalize_data(df, col_name):
        data = df[col_name].apply(json.loads)
        return pd.json_normalize(data)

    def clean_data(self):
        """
        Cleaning process of data:
        1- normalize
        2- deleting useless state (GRACE, PENDING, AND PARTIALLY PAID) on data
        """
        expected = Processing_data.normalize_data(self.data, 'expected_detail')
        paid = Processing_data.normalize_data(self.data, 'paid_detail')
        self.data = self.data.join(expected).join(paid, rsuffix='_paid')
        self.data = self.data[self.data.state != "GRACE"]
        self.data = self.data[self.data.state != "PENDING"]
        self.data = self.data[self.data.state != "PARTIALLY_PAID"]
        return self.data

    def delete_unnecessary_columns(self):
        """Deletes unnecesary columns produced by the json format"""
        data_droped_columns = ['penalty', 'interest', 'insurance', 'principal', 'taxRetentionValue',
                               'taxRetentionPercentage', 'legalCharge', 'preLegalCharge', 'feesPaid',
                               'fngTotal', 'fngValue',
                               'fngPercent', 'fngVatValue', 'fngVatPercent', 'monthlyDeferredInterest',
                               'penalty_paid', 'interest_paid', 'insurance_paid',
                               'principal_paid', 'taxRetentionValue_paid',
                               'taxRetentionPercentage_paid', 'legalCharge_paid',
                               'preLegalCharge_paid', 'feesPaid_paid', 'fngTotal_paid',
                               'fngValue_paid', 'fngPercent_paid', 'fngVatValue_paid',
                               'fngVatPercent_paid', 'monthlyDeferredInterest_paid', 'expected_detail', 'paid_detail']
        self.data = self.data.drop(columns=data_droped_columns)
        return self.data

    def organizing_data(self):
        """
        Organize data after having the first draft of your dataframe and fills total na rows with 0
        and groups clients by their six newest loan_id
        This is step three
        """
        self.data = self.data.sort_values(
            ['client_id', 'loan_id', 'expected_date'])
        self.data['total'] = self.data['total'].fillna(0)
        self.data['total_paid'] = self.data['total_paid'].fillna(0)
        self.data = self.data.groupby('loan_id').filter(
            lambda x: x['loan_id'].value_counts() > 6).groupby('loan_id').tail(6)
        return self.data

    def checking_cat_data(self):
        """Checks for the clients that are present in the two tables"""
        self.cat_data = self.cat_data.sort_values(
            by='ID Cliente', ascending=True)

        rep_columns = ['Procesos judiciales', 'Alertas', 'Website empresa',
                       'Instagram empresa', 'LinkedIn empresa',
                       'LinkedIn empresarios', 'Impacto', 'Acceso previso a la banca',
                       'Mujeres empresarias']

        # Replace values for Si/No
        self.cat_data[rep_columns] = self.cat_data[rep_columns].replace({'No procesos judiciales': 'No',
                                                                         'Sí procesos judiciales': 'Si',
                                                                         'No Alertas': 'No',
                                                                         'Sí Alertas': 'Si',
                                                                         'No website': 'No',
                                                                         'Si website': 'Si',
                                                                         'No Ig': 'No',
                                                                         'Si Ig': 'Si',
                                                                         'No LinkedIn': 'No',
                                                                         'Si LinkedIn': 'Si',
                                                                         'No LinkedIn empresarios': 'No',
                                                                         'Si LinkedIn empresarios': 'Si',
                                                                         'Si Impacto': 'Si',
                                                                         'No Impacto': 'No',
                                                                         'Si acceso a la banca': 'Si',
                                                                         'No acceso a la banca': 'No',
                                                                         'No mujeres empresarias': 'No',
                                                                         'Si mujeres empresarias': 'Si'})
        return self.cat_data

    def deleting_cat_info(self):
        """Deletes cat_data to analize clients on mean"""
        cat_cols = ['ID Cliente', 'Monto Acumulado', 'Uso de los recursos', 'Plazo',
                    'Sector', 'Ingresos', 'Ubicación', 'Estrato Mínimo',
                    'Procesos judiciales', 'Alertas', 'Score Bureau Empresa', 'Huellas de Consulta', 'Tiempo en el negocio',
                    'Website empresa', 'Instagram empresa', 'LinkedIn empresa',
                    'LinkedIn empresarios', 'Edad empresarios', 'Activador', 'Número de accionistas',
                    'Impacto', 'Acceso previso a la banca', '# Empleados',
                    'Mujeres empresarias', 'Mujeres en cargos directivos']

        self.data = self.data.drop(columns=cat_cols)
        return self.data

    def merge_two_tables(self):
        """Merging two tables: Numerical and cathegorical"""
        self.data = self.data.merge(
            self.cat_data, left_on='client_id', right_on='ID Cliente')
        return self.data

    @staticmethod
    def red_state(data):
        count = 0
        cond = 1
        for i, value in enumerate(data):
            if value == 0:
                count += 1
            if i >= 4:
                if value >= 15:
                    cond = 0
                
        if count <= 2 and cond == 0 and sum(data) > 30:
            return 'LATE'
        elif sum(data) > 30:
            return 'UNDEF'
        else:
            return 'PAID'
    
    def customer_level(self):
        """Compress dataframe into a better study"""
        self.data = self.data.groupby(['client_id', 'loan_id']).agg({
            'state': lambda x: x.iloc[-1],  # Devuelve el último state
            'arrears_days': lambda x:  list(x),
            'total_paid': 'sum'}).round(0)  # Se suman todas las facturas
        
        self.data['state'] = self.data['arrears_days'].apply(Processing_data.red_state)
        self.data['arrears_days'] = self.data['arrears_days'].apply(lambda x: sum(x))
        self.data.reset_index(inplace=True)
        return self.data

    def head(self):
        """print head of df"""
        return self.data.head()

    # Funcion compacta de limpieza de data
    def data_cleaning(self):
        """This function resumes all the steps of organizing and cleaning the datasets"""
        self.data = self.fill_na_json()
        self.data = self.clean_data()
        self.data = self.delete_unnecessary_columns()
        self.data = self.organizing_data()
        self.cat_data = self.checking_cat_data()
        self.data = self.merge_two_tables()
        self.data = self.deleting_cat_info()
        self.data = self.customer_level()
        self.data = self.merge_two_tables()
        return self.data
        
# ---------------------------------------

    def transform_data(self):
        """transform_data : Transforma los datos numericos con el metodo PowerTransformer"""
        self.data = self.data_cleaning()
        #if self.cleaning == True:
        #    self.data = self.data_cleaning()
        #else:
        #    self.cat_data["ID Cliente"] = self.data['client_id']
        #    self.data.drop(["ID Cliente"], axis= 1, inplace=True)
        #    self.data = self.merge_two_tables()
            
        for c in self.data.select_dtypes(exclude='object').columns:
            pt = PowerTransformer()
            self.data[c] = pt.fit_transform(
                np.array(self.data[c]).reshape(-1, 1))
        return self.data

    def reduction_dim(self):
        """reduction: Reduce la dimensionalidad de los datos aplicando Analisis Factorial de Datos Mixtos(FAMD)"""
        self.data = self.transform_data()
        
        self.data['state'] = self.data['state'].replace(to_replace="LATE",value="0")
        self.data['state'] = self.data['state'].replace(to_replace="PAID",value="1")
        self.data['state'] = self.data['state'].astype(object)

        # Declarar metodo para aplicar FAMD
        famd = prince.FAMD(
            n_components=2,
            n_iter=3,
            copy=True,
            check_input=True,
            engine='auto',
            random_state=42)

        # Ajustar y transformar la dimensión aplicando FAMD
        famd = famd.fit(self.data)
        transformada = famd.transform(self.data)

        Y = transformada.to_numpy()
        principalDf_famd = pd.DataFrame(
            data=Y, columns=['principal component 1', 'principal component 2'])
        finalDf_Cat_famd = pd.concat(
            [principalDf_famd, self.data['state']], axis=1, ignore_index=True)

        self.data = finalDf_Cat_famd
        return self.data

    def execute_model(self):
        """execute: Función que ejecuta el modelo con los datos procesados"""
        self.data = self.reduction_dim().to_numpy()
        self.kproto = KPrototypes(n_clusters=2, init = 'Huang', max_iter=100, verbose = 1,
                                  n_init = 15, random_state=4444, n_jobs=-1, gamma=.25)
        self.clusters = self.kproto.fit_predict(self.data, categorical=[2])
        #self.kproto = kproto
        print(self.kproto)
        #print(self.clusters)
        return self.kproto

    def guardar_model(self):
        """Saves the model into a joblib file"""
        try:
            joblib.dump(self.execute_model(), 'testeo.joblib')
            print("Saved correctly!")
        except:
            print("There is a problem to save the model, check on documentation")


class Testing_model(Processing_data):
    """This class test the model"""

    def __init__(self, data):
        """Testing model constructor"""
        #super(kproto, clusters).__init__()
        self.data = data
        self.kproto = []
        self.clusters = []
    
    def transform_data(self):
        """Transform data"""
        print("here")
        #self.data.replace([np.inf, -np.inf], np.nan, inplace=True)
        self.data.drop(["client_id",'loan_id', 'ID Cliente'], axis= 1, inplace=True)
        for c in self.data.select_dtypes(exclude='object').columns:
            pt = PowerTransformer()
            self.data[c] = pt.fit_transform(
                np.array(self.data[c]).reshape(-1, 1))
            print(self.data[c])
        #self.data.fillna(self.data.mean())
        #print(self.data.isnull().sum())
        return self.data


In [8]:
test = Processing_data(data, cat_data)

In [9]:
counts = test.data_cleaning()

In [10]:
counts['state'].value_counts()

LATE    306
PAID    271
Name: state, dtype: int64

In [11]:
counts.to_csv('second_option.csv')

In [10]:
new_data = Processing_data(data, cat_data)

In [8]:
new_data.execute_model()

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


Best run was number 3
KPrototypes(gamma=0.25, n_clusters=2, n_init=15, n_jobs=-1, random_state=4444,
            verbose=1)


KPrototypes(gamma=0.25, n_clusters=2, n_init=15, n_jobs=-1, random_state=4444,
            verbose=1)

In [11]:
%%time
new_data.guardar_model()

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


Best run was number 3
KPrototypes(gamma=0.25, n_clusters=2, n_init=15, n_jobs=-1, random_state=4444,
            verbose=1)
Saved correctly!
Wall time: 10.8 s


In [62]:
data_544 = data_prueba.reset_index()
data_544[data_544['index'] == 544]

,index,client_id,loan_id,state,arrears_days,total_paid,ID Cliente,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
544,544,5692,1399,LATE,63,95941569.0,5692.0,420000000,KT - EX,13 a 24 meses,Servicios,Scale Up,Bogotá D.C.,4,No,No,925.0,8.0,2010,Si,No,No,No,42,Internet,2.0,No,Si,52,No,3


In [61]:
prueba

,state,arrears_days,total_paid,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
544,0,1.230459,0.0,0.0,KT - EX,13 a 24 meses,Servicios,Scale Up,Bogotá D.C.,0.328725,No,No,0.536097,1.053392,-0.617804,Si,No,No,No,0.545512,Internet,-0.003568,No,Si,1.342100,No,1.182480
421,0,0.151593,0.0,0.0,KT - EX,13 a 24 meses,Servicios,Scale Up,Bogotá D.C.,-1.076472,No,No,0.398698,-0.078928,0.821658,Si,Si,Si,Si,-0.248540,Internet,-1.861879,No,No,0.734209,No,1.182480
236,1,-0.991265,0.0,0.0,SP,25 a 36 meses,Servicios,Seed,Bogotá D.C.,1.285983,No,Si,0.573548,1.421122,-1.342021,Si,No,No,Si,0.121905,Internet,-0.003568,No,Si,1.042463,No,0.396612
466,1,-1.485704,0.0,0.0,KT - CO,Menos de 12 meses,Servicios,Seed,Bogotá D.C.,1.285983,No,No,-2.403806,-1.263546,1.046848,No,No,No,Si,-1.084582,Newspapers,-0.003568,No,No,-1.442847,No,-1.359368
449,1,-0.514328,0.0,0.0,SP,25 a 36 meses,Comercio,Seed,Marinilla,-1.076472,No,Si,0.548581,-0.509921,-1.342021,Si,Si,No,No,1.290748,Other,-0.003568,No,Si,-0.152999,No,0.396612
534,0,1.401992,0.0,0.0,KT - EX,13 a 24 meses,Servicios,Venture,Bogotá D.C.,-1.076472,Si,Si,0.379952,-1.263546,0.386492,Si,No,No,No,-1.651943,CommunicationMedia,1.879719,No,Si,-0.368966,Si,-0.439448
569,0,0.207252,0.0,0.0,KT - CO,13 a 24 meses,Comercio,Seed,Medellín,0.328725,Si,No,-0.033070,0.641427,1.046848,Si,No,No,No,1.026899,SocialNetworks,-0.003568,No,Si,-1.153960,No,-1.359368


In [65]:
# Se genera datos para prueba
new_data = Processing_data(data, cat_data)
data_prueba = new_data.data_cleaning()

# Cargar modelo
clf_model = joblib.load('testeo.joblib')

# Generando un muestra aleatoria de 20 clientes
prueba= data_prueba.sample(7)

In [66]:
prueba

,client_id,loan_id,state,arrears_days,total_paid,ID Cliente,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
284,2439,1188,PAID,0,17737257.0,2439.0,310000000,KT - EX,25 a 36 meses,Industria,Scale Up,Sabaneta,4,No,No,925.0,27.0,1993,Si,No,No,No,56,FriendOrColleague,5.0,No,Si,120,No,25
482,4409,1287,LATE,187,6275878.0,4409.0,30000000,KT - EX,13 a 24 meses,Servicios,Seed,Bogotá D.C.,4,No,No,895.0,1.0,2013,No,No,No,No,58,FriendOrColleague,2.0,No,No,3,Si,1
318,2462,1116,PAID,1,216844031.0,2462.0,1230000000,CR,25 a 36 meses,Servicios,Venture,Medellín,6,No,No,933.0,16.0,2011,Si,Si,Si,Si,33,FriendOrColleague,2.0,No,Si,22,No,3
319,2464,626,PAID,1,43674921.0,2464.0,200000000,SP,25 a 36 meses,Servicios,Venture,Medellín,5,Si,No,929.0,4.0,2014,Si,Si,Si,Si,30,Other,2.0,No,Si,40,No,8
4,428,120,PAID,64,8102608.0,428.0,40000000,SP,13 a 24 meses,Industria,Seed,Medellín,4,No,No,861.0,1.0,2015,Si,No,Si,Si,44,CreditProfessionalContact,2.0,No,Si,5,No,0
229,1741,637,PAID,52,230633492.0,1741.0,731112570,KT - EX,Menos de 12 meses,Servicios,Scale Up,Bogotá D.C.,4,No,No,713.0,11.0,2013,Si,Si,Si,No,33,Other,6.0,No,No,15,No,0
369,2774,1429,LATE,83,12424684.0,2774.0,50000000,SP,Menos de 12 meses,Servicios,Seed,Bogotá D.C.,3,No,No,936.0,0.0,2002,No,No,No,No,52,Newspapers,3.0,No,Si,7,No,0


#### Testeando modelo

In [67]:
# Se genera datos para prueba
new_data = Processing_data(data, cat_data)
data_prueba = new_data.data_cleaning()

# Cargar modelo
clf_model = joblib.load('testeo.joblib')

# Generando un muestra aleatoria de 20 clientes
prueba= data_prueba.sample(7)
#numerica = prueba.select_dtypes(exclude='object')
#categorica = prueba.select_dtypes(exclude=['int64', 'float64'])

#Procesar los datos de prueba
data_modelo = Testing_model(prueba)

#data_modelo.drop(['client_id', 'loan_id'], axis=1, inplace=True)
entrenar = data_modelo.reduction_dim().to_numpy()

print(f'clientes a evaluar {len(entrenar)}\n')
for i in range(len(entrenar)):
    print(f'cliente #{i} pertence al cluster: {clf_model.predict(entrenar, categorical=[2])[i]}')

here
122    0.816650
355   -1.223001
261   -0.187479
165    0.892390
35    -1.492862
259   -0.132241
499    1.326544
Name: arrears_days, dtype: float64
122    0.062276
355    0.680641
261    1.404666
165   -0.193054
35     0.300251
259   -0.155196
499   -2.099585
Name: total_paid, dtype: float64
122    0.840030
355   -0.077103
261    1.312540
165   -1.575177
35     0.915944
259   -0.359802
499   -1.056433
Name: Monto Acumulado, dtype: float64
122    0.632456
355    0.632456
261    0.632456
165   -1.581139
35     0.632456
259    0.632456
499   -1.581139
Name: Estrato Mínimo, dtype: float64
122   -0.650510
355    0.831811
261    0.536219
165   -2.204438
35     0.480917
259    0.519576
499    0.486425
Name: Score Bureau Empresa, dtype: float64
122    1.193663
355   -1.682361
261    1.470156
165   -0.414346
35     0.261580
259   -0.414346
499   -0.414346
Name: Huellas de Consulta, dtype: float64


overflow encountered in power


ValueError: Input contains infinity or a value too large for dtype('float64').

In [68]:
prueba

,state,arrears_days,total_paid,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
122,LATE,0.816650,0.062276,0.840030,KT - EX,13 a 24 meses,Industria,Venture,Itagüí,0.632456,No,No,-0.650510,1.193663,2011,Si,Si,No,Si,53,Other,2.0,No,Si,10,No,2
355,PAID,-1.223001,0.680641,-0.077103,KT - EX,25 a 36 meses,Comercio,Seed,Bogotá D.C.,0.632456,No,No,0.831811,-1.682361,2014,No,No,No,Si,36,FriendOrColleague,19.0,No,Si,5,No,1
261,PAID,-0.187479,1.404666,1.312540,KT - EX,Menos de 12 meses,Servicios,Venture,Medellín,0.632456,No,Si,0.536219,1.470156,2013,No,Si,Si,Si,30,FriendOrColleague,2.0,No,Si,10,No,1
165,PAID,0.892390,-0.193054,-1.575177,CR,13 a 24 meses,Industria,Venture,Medellín,-1.581139,No,No,-2.204438,-0.414346,2014,Si,Si,Si,Si,45,Internet,1.0,Si,No,10,No,0
35,PAID,-1.492862,0.300251,0.915944,KT - EX,Menos de 12 meses,Servicios,Scale Up,Bogotá D.C.,0.632456,No,No,0.480917,0.261580,2013,Si,No,No,No,37,Other,4.0,No,Si,32,No,3
259,PAID,-0.132241,-0.155196,-0.359802,KT - EX,Menos de 12 meses,Servicios,Seed,Bogotá D.C.,0.632456,No,Si,0.519576,-0.414346,2013,No,No,No,Si,33,FriendOrColleague,3.0,Si,No,30,No,1
499,LATE,1.326544,-2.099585,-1.056433,KT - EX,13 a 24 meses,Servicios,Scale Up,Bogotá D.C.,-1.581139,No,Si,0.486425,-0.414346,2014,No,Si,No,No,54,Other,1.0,No,Si,5,No,0
